In [2]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import tensorflow as tf
# !pip install transformers
from transformers import BertTokenizer

2023-02-11 16:10:34.157582: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [20]:
# None=0, Seek=1, Provide=2

df = pd.read_csv("CSN.csv")
df

,text,label
0,I don't know who left other than him.,0
1,I also was very depressed and cryed very very ...,1
2,I am a 6 yr survivor and have been out of trea...,0
3,I have enough energy for 16 hour periods.,0
4,I was diagnosed in October 2009 at age 53.,0
...,...,...
5002,"You are a creative person, and I can just imag...",0
5003,I thought they were supposed to eliminate the ...,0
5004,"I could use a good hug too, from someone who u...",1
5005,"You take 1 pill one hour prior to treatment, 1...",0


In [ ]:
# df_0 = df.loc[df['label']==0]
# df_1 = df[df['label']==1]
# # df_2 = df[df['level']==2]

# df_0_downsampled = df_0.sample(df_1.shape[0])
# # df_2_downsampled = df_2.sample(df_1.shape[0])

# # df = pd.concat([df_0_downsampled, df_1, df_2_downsampled])
# df = pd.concat([df_0_downsampled, df_1])
# df

In [57]:
df_seek = df.copy()
print(df_seek.groupby('label').describe())
df_response = df.copy()
print(df_response.groupby('label').describe())
df_seek.loc[df_seek['label'] == 2, 'label'] = 0
print(df_seek.groupby('label').describe())
df_response.loc[df_response['label'] == 1, 'label'] = 0
print(df_response.groupby('label').describe())
df_response.loc[df_response['label'] == 2, 'label'] = 1
print(df_response.groupby('label').describe())

       text                                                               
      count unique                                                top freq
label                                                                     
0      2995   2995              I don't know who left other than him.    1
1      1046   1046  I also was very depressed and cryed very very ...    1
2       966    966  I just wanted to let you know I was here if yo...    1
       text                                                               
      count unique                                                top freq
label                                                                     
0      2995   2995              I don't know who left other than him.    1
1      1046   1046  I also was very depressed and cryed very very ...    1
2       966    966  I just wanted to let you know I was here if yo...    1
       text                                                               
      count unique       

TRAINING SEEKER SIDE ON CSN

In [58]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

In [60]:
token = tokenizer.encode_plus(
    df_seek['text'].iloc[0], 
    max_length=256, 
    truncation=True, 
    padding='max_length', 
    add_special_tokens=True,
    return_tensors='tf'
)

In [61]:
X_input_ids = np.zeros((len(df), 256))
X_attn_masks = np.zeros((len(df), 256))

In [62]:
def generate_training_data(df_balanced, ids, masks, tokenizer):
    for i, text in tqdm(enumerate(df_seek['text'])):
        tokenized_text = tokenizer.encode_plus(
            text,
            max_length=256, 
            truncation=True, 
            padding='max_length', 
            add_special_tokens=True,
            return_tensors='tf'
        )
        ids[i, :] = tokenized_text.input_ids
        masks[i, :] = tokenized_text.attention_mask
    return ids, masks

In [63]:
X_input_ids, X_attn_masks = generate_training_data(df, X_input_ids, X_attn_masks, tokenizer)

0it [00:00, ?it/s]

In [66]:
labels = np.zeros((len(df_seek), 2))
labels.shape

(5007, 2)

In [67]:
labels[np.arange(len(df_seek)), df_seek['label'].values] = 1 # one-hot encoded target te

In [68]:
# creating a data pipeline using tensorflow dataset utility, creates batches of data for easy loading...
dataset = tf.data.Dataset.from_tensor_slices((X_input_ids, X_attn_masks, labels))
dataset.take(1) # one sample data

<TakeDataset element_spec=(TensorSpec(shape=(256,), dtype=tf.float64, name=None), TensorSpec(shape=(256,), dtype=tf.float64, name=None), TensorSpec(shape=(2,), dtype=tf.float64, name=None))>

In [69]:
def SentimentDatasetMapFunction(input_ids, attn_masks, labels):
    return {
        'input_ids': input_ids,
        'attention_mask': attn_masks
    }, labels

In [70]:
dataset = dataset.map(SentimentDatasetMapFunction) # converting to required format for tensorflow dataset 

In [71]:
dataset = dataset.shuffle(10000).batch(16, drop_remainder=True) # batch size, drop any left out tensor

In [72]:
p = 0.8
train_size = int((len(df_seek)//16)*p) # for each 16 batch of data we will have len(df)//16 samples, take 80% of that for train.

In [73]:
train_dataset = dataset.take(train_size)
val_dataset = dataset.skip(train_size)

In [74]:
train_dataset

<TakeDataset element_spec=({'input_ids': TensorSpec(shape=(16, 256), dtype=tf.float64, name=None), 'attention_mask': TensorSpec(shape=(16, 256), dtype=tf.float64, name=None)}, TensorSpec(shape=(16, 2), dtype=tf.float64, name=None))>

In [75]:
from transformers import TFBertModel

In [76]:
model = TFBertModel.from_pretrained('bert-base-cased') # bert base model with pretrained weights

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [77]:
# defining 2 input layers for input_ids and attn_masks
input_ids = tf.keras.layers.Input(shape=(256,), name='input_ids', dtype='int32')
attn_masks = tf.keras.layers.Input(shape=(256,), name='attention_mask', dtype='int32')

bert_embds = model.bert(input_ids, attention_mask=attn_masks)[1] # 0 -> activation layer (3D), 1 -> pooled output layer (2D)
intermediate_layer = tf.keras.layers.Dense(512, activation='relu', name='intermediate_layer')(bert_embds)
output_layer = tf.keras.layers.Dense(2, activation='softmax', name='output_layer')(intermediate_layer) # softmax -> calcs probs of classes

sentiment_model = tf.keras.Model(inputs=[input_ids, attn_masks], outputs=output_layer)
sentiment_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 256)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 256)]        0           []                               
                                                                                                  
 bert (TFBertMainLayer)         TFBaseModelOutputWi  108310272   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 256,                                           

In [78]:
optim = tf.keras.optimizers.legacy.Adam(learning_rate=1e-5, decay=1e-6)
loss_func = tf.keras.losses.CategoricalCrossentropy()
acc = tf.keras.metrics.CategoricalAccuracy('accuracy')

In [79]:
sentiment_model.compile(optimizer=optim, loss=loss_func, metrics=[acc])

In [80]:
hist = sentiment_model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=2
)

Epoch 1/2


2023-02-06 20:11:17.160977: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


249/249 [==============================] - ETA: 0s - loss: 0.4304 - accuracy: 0.8183

2023-02-06 20:16:39.680945: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


249/249 [==============================] - 352s 1s/step - loss: 0.4304 - accuracy: 0.8183 - val_loss: 0.3211 - val_accuracy: 0.8562
Epoch 2/2
249/249 [==============================] - 332s 1s/step - loss: 0.2990 - accuracy: 0.8715 - val_loss: 0.1909 - val_accuracy: 0.9355


In [82]:
sentiment_model.save('sentiment_model_seek')

INFO:tensorflow:Assets written to: sentiment_model_seek/assets


INFO:tensorflow:Assets written to: sentiment_model_seek/assets


In [10]:
sentiment_model_seek = tf.keras.models.load_model('sentiment_model_seek')

tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

def prepare_data(input_text, tokenizer):
    token = tokenizer.encode_plus(
        input_text,
        max_length=256, 
        truncation=True, 
        padding='max_length', 
        add_special_tokens=True,
        return_tensors='tf'
    )
    return {
        'input_ids': tf.cast(token.input_ids, tf.float64),
        'attention_mask': tf.cast(token.attention_mask, tf.float64)
    }

def predict_class(processed_data):
    '''predict class of input text
  Args:
    - reviews (list of strings)
  Output:
    - class (list of int)
  '''
    result = []
    for i in processed_data:
        pred = sentiment_model_seek.predict(i)
        result.append(np.argmax(pred))
    return result
#     print(sentiment_model_seek.predict(processed_data))
#     return [np.argmax(pred) for pred in sentiment_model_seek.predict(processed_data)]
#     probs = model.predict(processed_data)[0]
#     return classes[np.argmax(probs)]



2023-02-11 16:17:50.188478: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-11 16:17:50.193066: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-02-11 16:17:50.193508: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1 Pro


In [25]:
df_self = pd.read_csv("self_eval_seek_2.csv")
df_self["label"].value_counts()
df_self = df_self.loc[:, ~df_self.columns.str.contains('^Unnamed')]
df_self = df_self[df_self["label"].notna()]
df_self.loc[df_self["label"]=="Seeking(subtly)", "label"] = int(0)
df_self.loc[df_self['label']=="Not Seeking", "label"] = int(0)
df_self.loc[df_self['label']=="Not Seeking/Maybe", "label"] = int(0)
df_self.loc[df_self['label']=="Seeking(truly)", "label"] = int(1)
df_self['label'].astype(int)
# df_self
# df_self = df_self[df_self['label']=="Seeking(subtly)", "label"] = 0
# df_self = df_self[df_self['label']=="Seeking(subtly)", "label"] = 0

0      0
1      1
2      0
3      1
4      1
      ..
196    1
197    0
198    1
199    1
200    1
Name: label, Length: 201, dtype: int64

In [12]:
from sklearn.metrics import classification_report

processed_data = []

for i in df_self["seeker_post"]:
    processed_data.append(prepare_data(i, tokenizer))
# print(processed_data)
y_pred = predict_class(processed_data)
print(y_pred)


2023-02-11 16:18:17.462859: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 141ms/step
[0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1]


ValueError: Classification metrics can't handle a mix of unknown and binary targets

In [27]:
print(classification_report(df_self['label'].to_list(), y_pred))
# df_self['label'].astype(int)
# print(len(df_self['label']), len(y_pred))

              precision    recall  f1-score   support

           0       0.64      0.52      0.57        75
           1       0.74      0.83      0.78       126

    accuracy                           0.71       201
   macro avg       0.69      0.67      0.68       201
weighted avg       0.70      0.71      0.70       201

